## master main

In [ ]:
# main.py
import os
import datetime
import json
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

from src.config.settings import BASE_CONFIG
from src.config.config_generator import ConfigGenerator
from src.data.data_handler import DataHandler
from src.indicators.technical import Signal 
from src.simulation.simulator import Simulator
from src.utils.daily_results import DailyResults
from src.utils.performance import compute_monthly_rolling_sharpe 
from src.analysis.daily import DailyResultAnalyzer


class SimulationProject:
    def __init__(self, base_config):
        self.base_config = base_config
        data_handler = DataHandler(base_config)
        self.full_df = data_handler.load_data(saving_space=True)
        config_gen = ConfigGenerator(base_config, config_file="config_list.json")
        self.config_list = config_gen.get_configs()
        self.results_folder = "results"
        if not os.path.exists(self.results_folder):
            os.makedirs(self.results_folder)
        self.plot_enabled = base_config.get("plot", False)
        if self.plot_enabled:
            self.pdf = PdfPages("%s_my_%s_%s.pdf" % (base_config['ticker'], base_config['longshort'],
                                                      datetime.datetime.now().strftime("%m%d")))
        else:
            self.pdf = None
        self.simulator = Simulator(self.full_df)
        self.daily_analyzer = DailyResultAnalyzer(results_folder=self.results_folder,
                                                  plot_enabled=self.plot_enabled,
                                                  pdf=self.pdf)

    def run(self):
        for cfg in self.config_list:

            signal_config = {
                "signal_type": cfg.get("signal_type", "RSI"),
                "window": cfg["window"],
                "threshold": cfg["threshold"],
                "longshort": cfg["longshort"]
            }
            signal_obj = Signal.create(self.full_df, signal_config)
            signals = signal_obj.get_signal()
            pnl_gross, cost, slippage, pos = self.simulator.simulate(
                signals,
                hold_period=cfg["hold_period"],
                num_of_share=cfg["num_of_share"],
                cost_ratio=cfg["cost_ratio"],
                slippage_ratio=cfg["slippage_ratio"]
            )
            daily_results = DailyResults.get_daily_results_df(self.full_df, pnl_gross, cost, slippage, pos)
            config_key = "Hold%s_%s_Thr%s" % (cfg["hold_period"], cfg["window"], cfg["threshold"])
            self.daily_analyzer.process_and_save(self.full_df, daily_results, config_key)
        if self.plot_enabled:
            self.daily_analyzer.finalize_plots()

def main():
    project = SimulationProject(BASE_CONFIG)
    project.run()

if __name__ == "__main__":
    main()


Processed: hold_period=10, window=3, threshold=5 (saved to results\Hold10_3_Thr5.csv)
Processed: hold_period=10, window=3, threshold=10 (saved to results\Hold10_3_Thr10.csv)
Processed: hold_period=10, window=3, threshold=20 (saved to results\Hold10_3_Thr20.csv)
Processed: hold_period=10, window=3, threshold=30 (saved to results\Hold10_3_Thr30.csv)
Processed: hold_period=30, window=3, threshold=5 (saved to results\Hold30_3_Thr5.csv)
Processed: hold_period=30, window=3, threshold=10 (saved to results\Hold30_3_Thr10.csv)
Processed: hold_period=30, window=3, threshold=20 (saved to results\Hold30_3_Thr20.csv)
Processed: hold_period=30, window=3, threshold=30 (saved to results\Hold30_3_Thr30.csv)
Processed: hold_period=60, window=3, threshold=5 (saved to results\Hold60_3_Thr5.csv)
Processed: hold_period=60, window=3, threshold=10 (saved to results\Hold60_3_Thr10.csv)
Processed: hold_period=60, window=3, threshold=20 (saved to results\Hold60_3_Thr20.csv)
Processed: hold_period=60, window=3, t

## parallel

In [ ]:
# main.py
import os
import datetime
import itertools
import json
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import multiprocessing

from src.config.settings import BASE_CONFIG
from src.config.config_generator import ConfigGenerator
from src.data.data_handler import DataHandler
from src.indicators.technical import Signal  # Factory parent class
from src.simulation.simulator import Simulator
from src.strategies.simple_strategy import SimpleStrategy
from src.utils.daily_results import DailyResults
from src.utils.performance import compute_monthly_rolling_sharpe 
from src.analysis.daily import DailyResultAnalyzer

class SimulationProject:
    def __init__(self, base_config):
        self.base_config = base_config
        data_handler = DataHandler(base_config)
        self.full_df = data_handler.load_data(saving_space=True)
        config_gen = ConfigGenerator(base_config, config_file="config_list.json")
        self.config_list = config_gen.get_configs()
        self.results_folder = "results_parallel"
        if not os.path.exists(self.results_folder):
            os.makedirs(self.results_folder)
        self.plot_enabled = base_config.get("plot", False)
        self.pdf = None
        if self.plot_enabled:
            self.pdf = PdfPages("%s_my_%s_%s.pdf" % (base_config['ticker'],
                                                     base_config['longshort'],
                                                     datetime.datetime.now().strftime("%m%d")))
        self.simulator = Simulator(self.full_df)
        self.daily_analyzer = DailyResultAnalyzer(results_folder=self.results_folder,
                                                  plot_enabled=self.plot_enabled,
                                                  pdf=self.pdf)

    @staticmethod
    def init_globals(full_df, simulator, daily_analyzer):
        # Global variables accessible in each worker process
        global FULL_DF, SIMULATOR, DAILY_ANALYZER
        FULL_DF = full_df
        SIMULATOR = simulator
        DAILY_ANALYZER = daily_analyzer

    @staticmethod
    def task(cfg):
        signal_config = {
            "signal_type": cfg.get("signal_type", "RSI"),
            "window": cfg["window"],
            "threshold": cfg["threshold"],
            "longshort": cfg["longshort"]
        }
        # Create the signal object using the factory design.
        signal_obj = Signal.create(FULL_DF, signal_config)
        signals = signal_obj.get_signal()
        pnl_gross, cost, slippage, pos = SIMULATOR.simulate(
            signals,
            hold_period=cfg["hold_period"],
            num_of_share=cfg["num_of_share"],
            cost_ratio=cfg["cost_ratio"],
            slippage_ratio=cfg["slippage_ratio"]
        )
        daily_results = DailyResults.get_daily_results_df(FULL_DF, pnl_gross, cost, slippage, pos)
        config_key = "Hold%s_%s_Thr%s" % (cfg["hold_period"], cfg["window"], cfg["threshold"])
        DAILY_ANALYZER.process_and_save(FULL_DF, daily_results, config_key)
        return config_key

    def run(self):
        # Use multiprocessing.Pool to parallelize simulation tasks.
        num_processes = min(4, len(self.config_list))
        with multiprocessing.Pool(processes=num_processes,
                                  initializer=SimulationProject.init_globals,
                                  initargs=(self.full_df, self.simulator, self.daily_analyzer)) as pool:
            results = pool.map(SimulationProject.task, self.config_list)
        if self.plot_enabled:
            self.daily_analyzer.finalize_plots()
        print("Processed configurations:", results)

if __name__ == "__main__":
    project = SimulationProject(BASE_CONFIG)
    project.run()


## composite strategy

In [6]:
# main2.py
import os
import datetime
import itertools
import random
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

from src.config.settings import BASE_CONFIG
from src.config.config_generator import ConfigGenerator
from src.data.data_handler import DataHandler
from src.indicators.technical import Signal  # Factory parent class
from src.simulation.simulator import Simulator
from src.strategies.composite_strategy import CompositeStrategy
from src.utils.daily_results import DailyResults
from src.utils.performance import compute_monthly_rolling_sharpe

class SimulationProject2:
    def __init__(self, base_config):
        self.base_config = base_config
        # Hyper-parameter: number of signals to combine.
        # This should be set in your BASE_CONFIG; default to 2 if not provided.
        self.num_of_signals = base_config.get("num_of_signals", 2)
        
        # Load and process data with precomputed signal columns.
        data_handler = DataHandler(base_config)
        self.full_df = data_handler.process_data(saving_space=True)
        
        # Generate configuration combinations from config_list.json.
        config_gen = ConfigGenerator(base_config, config_file="config_list.json")
        self.config_list = config_gen.get_configs()
        
        self.results_folder = "results2"
        if not os.path.exists(self.results_folder):
            os.makedirs(self.results_folder)
        
        self.plot_enabled = base_config.get("plot", False)
        if self.plot_enabled:
            pdf_filename = "%s_my_%s_%s.pdf" % (
                base_config['ticker'], base_config['longshort'],
                datetime.datetime.now().strftime("%m%d")
            )
            self.pdf = PdfPages(pdf_filename)
        
        self.simulator = Simulator(self.full_df)

    def run(self):
        # Create all unique combinations (order doesn't matter) of configurations of size num_of_signals.
        all_combinations = list(itertools.combinations(self.config_list, self.num_of_signals))
        
        # Randomly select up to 10 unique combinations.
        num_trials = 10
        if len(all_combinations) < num_trials:
            selected_combinations = all_combinations
        else:
            selected_combinations = random.sample(all_combinations, num_trials)
        
        for idx, cfg_combo in enumerate(selected_combinations):
            signals = []
            combo_key_parts = []
            
            # For each configuration in the combination, build the signal.
            for cfg in cfg_combo:
                signal_config = {
                    "signal_type": cfg.get("signal_type", "RSI"),
                    "window": cfg["window"],
                    "threshold": cfg["threshold"],
                    "longshort": cfg["longshort"]
                }
                signal_obj = Signal.create(self.full_df, signal_config)
                signal = signal_obj.get_signal()
                signals.append(signal)
                # Build a string key based on key parameters from each config.
                combo_key_parts.append("H%s_W%s_T%s" % (cfg["hold_period"], cfg["window"], cfg["threshold"]))
            
            # Use equal weights for composite strategy.
            weights = [1.0 / len(signals)] * len(signals)
            composite_strategy = CompositeStrategy(signals=signals, weights=weights, threshold=0)
            combined_signals = composite_strategy.get_combined_signal()
            
            # For simulation parameters, we'll use the first configuration of the combo.
            cfg0 = cfg_combo[0]
            pnl_gross, cost, slippage, pos = self.simulator.simulate(
                combined_signals,
                hold_period=cfg0["hold_period"],
                num_of_share=cfg0["num_of_share"],
                cost_ratio=cfg0["cost_ratio"],
                slippage_ratio=cfg0["slippage_ratio"]
            )
            
            daily_results = DailyResults.get_daily_results_df(self.full_df, pnl_gross, cost, slippage, pos)
            monthly_sharpe = compute_monthly_rolling_sharpe(daily_results["GrossPnL"])
            daily_results["MonthlySharpe"] = monthly_sharpe

            config_key = "_".join(combo_key_parts)
            csv_filename = os.path.join(self.results_folder, config_key + ".csv")
            daily_results.to_csv(csv_filename)
            
            if self.plot_enabled:
                fig = DailyResults.plot_daily_results(daily_results, "Composite: " + config_key)
                self.pdf.savefig(fig, bbox_inches="tight")
                plt.close(fig)
            
            print("Processed combination %d: %s (saved to %s)" % (idx+1, config_key, csv_filename))
        
        if self.plot_enabled:
            self.pdf.close()
            print("All plots saved.")

if __name__ == "__main__":
    project = SimulationProject2(BASE_CONFIG)
    project.run()


Processed combination 1: H60_W3_T20_H30_W7_T20 (saved to results2\H60_W3_T20_H30_W7_T20.csv)
Processed combination 2: H30_W14_T10_H60_W14_T5 (saved to results2\H30_W14_T10_H60_W14_T5.csv)
Processed combination 3: H30_W7_T20_H60_W7_T10 (saved to results2\H30_W7_T20_H60_W7_T10.csv)
Processed combination 4: H10_W7_T30_H30_W14_T30 (saved to results2\H10_W7_T30_H30_W14_T30.csv)
Processed combination 5: H60_W3_T5_H60_W7_T30 (saved to results2\H60_W3_T5_H60_W7_T30.csv)
Processed combination 6: H60_W3_T30_H10_W14_T5 (saved to results2\H60_W3_T30_H10_W14_T5.csv)
Processed combination 7: H10_W3_T10_H60_W14_T10 (saved to results2\H10_W3_T10_H60_W14_T10.csv)
Processed combination 8: H60_W3_T20_H30_W7_T5 (saved to results2\H60_W3_T20_H30_W7_T5.csv)
Processed combination 9: H10_W7_T10_H60_W14_T10 (saved to results2\H10_W7_T10_H60_W14_T10.csv)
Processed combination 10: H30_W7_T5_H30_W7_T30 (saved to results2\H30_W7_T5_H30_W7_T30.csv)


In [ ]:
import multiprocessing

def square(n):
    return n * n

if __name__ == '__main__':
    numbers = [1, 2, 3, 4, 5]

    # Create a pool of 4 worker processes
    with multiprocessing.Pool(processes=4) as pool:
        # Map the function to the numbers list
        results = pool.map(square, numbers)
    
    print("Squares:", results)

In [ ]:
## Composite strategy class that aggregates multiple indicator signals using weighted combination; inherits from BaseStrategy.
